In [10]:
import pandas as pd

In [11]:
rawdata=pd.read_csv('Energy_Balance_2018_11_20.csv', delimiter=";" )

In [12]:
def dutycyclemode(rawdata,percent_batt):
    import datetime
    batt=percent_batt*96
    Consumption_afterPV=[]
    Charge=[]
    for i in rawdata.index:
        if rawdata.loc[i,'Generation']>=rawdata.loc[i,'Consumption']:
            Charge.append(rawdata.loc[i,'Generation']-rawdata.loc[i,'Consumption'])
            Consumption_afterPV.append(0)
        else:
            Charge.append(0)
            Consumption_afterPV.append(rawdata.loc[i,'Consumption']-rawdata.loc[i,'Generation'])    

    rawdata['Charge']=Charge
    rawdata['Consumption_afterPV']=Consumption_afterPV

    totalcharge=rawdata['Charge'].sum()/4

    datanotstart=rawdata[rawdata.Generation==0]
    dataonlystart=rawdata[rawdata.Generation!=0]
    start_time=dataonlystart.index[0]
    consumption_night=rawdata.loc[:start_time,'Consumption'].sum()/4

    if totalcharge+batt>9600:
        discharge_only_at_night=1
        if batt-consumption_night>0.6*9600:
            batt_start=batt-consumption_night
        else:
            batt_start=0.6*9600
    else:
        discharge_only_at_night=0
        batt_start=percent_batt*96

    contactor=[]

    for i in rawdata[rawdata.index[0]:start_time].drop(start_time).index:
        if discharge_only_at_night==1:
            contactor.append(1)
        else:
            contactor.append(0)
            
    lastchargetime=rawdata[rawdata['Charge']!=0].index[-1]
    stop_time=lastchargetime+pd.Timedelta('15 min')
    remain=batt_start-9600*0.6
    overcharge=batt_start+totalcharge-9600
    
    for i in rawdata[start_time:lastchargetime].index:
        if rawdata.loc[i,'Charge'] > 0:
            contactor.append(1)
            remain += rawdata.loc[i,'Charge']/4
        elif overcharge <= 0:
            contactor.append(0)
        elif remain==0:
            contactor.append(0)
        else:
            contactor.append(1)
            overcharge=overcharge-(rawdata.loc[i,'Consumption']-rawdata.loc[i,'Generation'])/4

    for i in rawdata[stop_time:].index:
        contactor.append(0)
        
    rawdata['Contactor']=contactor
        
    battleft=(percent_batt-60)*96*4
    rawdata['Discharge']=rawdata['Contactor']*0
    for i in rawdata.index:
        if rawdata.loc[i,'Contactor']==1:
            if rawdata.loc[i,'Charge'] != 0:
                battleft=battleft+rawdata.loc[i,'Charge']
            else:
                if battleft >= rawdata.loc[i,'Consumption_afterPV']:
                    rawdata.loc[i,'Discharge']= rawdata.loc[i,'Consumption_afterPV']
                    battleft=battleft- rawdata.loc[i,'Consumption_afterPV']
                else:
                    rawdata.loc[i,'Discharge']=battleft
                    battleft=0
                    
                    
    checkcharge=(percent_batt-60)*96*4
    cclist=[]
    for i in rawdata.index:
        if rawdata.loc[i,'Contactor']==1:
            if rawdata.loc[i,'Charge'] != 0 :
                if checkcharge+rawdata.loc[i,'Charge']>=40*96*4:
                    rawdata.loc[i,'Charge']=40*96*4-checkcharge
                    checkcharge=40*96*4
                    cclist.append(checkcharge)
                else:
                    checkcharge+=rawdata.loc[i,'Charge']
                    cclist.append(checkcharge)
            elif rawdata.loc[i,'Discharge'] != 0:
                if checkcharge-rawdata.loc[i,'Discharge']<=0:
                    rawdata.loc[i,'Discharge']=checkcharge
                    checkcharge=0
                    cclist.append(checkcharge)
                else:    
                    checkcharge = checkcharge - rawdata.loc[i,'Discharge']
                    cclist.append(checkcharge)
            else:
                cclist.append(checkcharge)
        else:
            cclist.append(checkcharge)
    rawdata['Batt_above60%']=cclist
    newpercent_batt=percent_batt+(rawdata.Charge.sum()-rawdata.Discharge.sum())/4/96
    return [rawdata,newpercent_batt]

In [13]:
def clean1(path):
    rawdata=pd.read_csv(path+".csv", delimiter=";" )
    deleteindex=rawdata.loc[rawdata.loc[pd.IndexSlice[:,'Battery discharging / Mean values [W] ']]==' '].index
    rawdata.drop(deleteindex,inplace=True)
    rawdata.rename(columns={' ':'Time'},inplace=True)
    rawdata['Time']=rawdata['Time'].str.replace('[^\d]','')
    rawdata['Time']=path[15:19]+path[20:22]+path[23:]+rawdata['Time']
    rawdata['Time']=pd.to_datetime(rawdata.Time)
    rawdata.set_index('Time', inplace=True)
    for i in rawdata.columns:
        if i != 'Direct consumption / Mean values [W] ' and i != 'Battery discharging / Mean values [W] ' and i!='External energy supply / Mean values [W] ' and i != 'PV power generation / Mean values [W]  ':
            rawdata.drop([i],axis=1,inplace=True)
    for i in rawdata.columns:
        rawdata[i].fillna('0',inplace=True)
        rawdata[i]=rawdata[i].apply(lambda x: x.replace(' ', '0'))
        rawdata[i]=rawdata[i].str.replace('\D','').astype(int)
    rawdata['Consumption']=rawdata['Direct consumption / Mean values [W] ']+rawdata['Battery discharging / Mean values [W] ']+rawdata['External energy supply / Mean values [W] ']
    rawdata.rename(columns={'PV power generation / Mean values [W]  ':'Generation'},inplace=True)
    rawdata.drop(['Direct consumption / Mean values [W] ', 'Battery discharging / Mean values [W] ','External energy supply / Mean values [W] '],axis=1,inplace=True)
    return rawdata

In [14]:
rawdata=clean1('Energy_Balance_2018_11_20')

In [15]:
rawdata

,Generation,Consumption
Time,,
2018-11-20 00:15:00,0,1709
2018-11-20 00:30:00,0,2226
2018-11-20 00:45:00,0,1610
2018-11-20 01:00:00,0,2030
2018-11-20 01:15:00,0,1840
2018-11-20 01:30:00,0,1708
2018-11-20 01:45:00,0,1954
2018-11-20 02:00:00,0,1525
2018-11-20 02:15:00,0,1953


In [23]:
dutycyclemode(rawdata,100)[0].Contactor.dropindex

AttributeError: 'Series' object has no attribute 'dropindex'